***
# 数据抓取：

> # 使用Python编写网络爬虫
***

王成军

wangchengjun@nju.edu.cn

计算传播网 http://computational-communication.com

# 需要解决的问题 

- 页面解析
- 获取Javascript隐藏源数据
- 自动翻页
- 自动登录
- 连接API接口


In [258]:
import requests
from bs4 import BeautifulSoup

- 一般的数据抓取，使用urllib2和beautifulsoup配合就可以了。
- 尤其是对于翻页时url出现规则变化的网页，只需要处理规则化的url就可以了。
- 以简单的例子是抓取天涯论坛上关于某一个关键词的帖子。
    - 在天涯论坛，关于雾霾的帖子的第一页是：
http://bbs.tianya.cn/list.jsp?item=free&nextid=0&order=8&k=雾霾
    - 第二页是：
http://bbs.tianya.cn/list.jsp?item=free&nextid=1&order=8&k=雾霾


***
# 数据抓取：
> # 抓取天涯回帖网络
***

王成军

wangchengjun@nju.edu.cn

计算传播网 http://computational-communication.com

In [257]:
from IPython.display import display_html, HTML
HTML('<iframe src= http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=PX width=1000 height=500></iframe>')
# the webpage we would like to crawl

In [259]:
page_num = 0
url = "http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=PX"% page_num
content = requests.get(url).text 
soup = BeautifulSoup(content, "lxml") 
articles = soup.find_all('tr')
#td[0].find('a', {'class', ""})

In [260]:
articles[0]

<tr>
<th scope="col"> 标题</th>
<th scope="col">作者</th>
<th scope="col">点击</th>
<th scope="col">回复</th>
<th scope="col">发表时间</th>
</tr>

In [261]:
articles[1]

<tr class="bg">
<td class="td-title facediamond">
<a class="diamond" href="http://bei.tianya.cn/wallet/zuan.do" target="_blank" title="钻石榜"></a>
<a class="" href="http://bbs.tianya.cn/post-free-5884168-1.shtml" target="_blank">（续）检举揭发现任中国石油天然气集团公司董事长党组书记王宜林行贿受贿</a>
</td>
<td><a class="author" href="http://www.tianya.cn/134712932" target="_blank">涛声依旧1232</a></td>
<td>6872</td>
<td>43</td>
<td title="null">04-10 14:15</td>
</tr>

In [180]:
len(articles[1:])

82

![](./img/inspect.png)

http://bbs.tianya.cn/list.jsp?item=free&nextid=0&order=8&k=PX
    
# 通过分析帖子列表的源代码，使用inspect方法，会发现所有要解析的内容都在‘td’这个层级下
    

In [181]:
for t in articles[1].find_all('td'): 
    print(t)

<td class="td-title facediamond">
<a class="diamond" href="http://bei.tianya.cn/wallet/zuan.do" target="_blank" title="钻石榜"></a>
<a class="" href="http://bbs.tianya.cn/post-free-5876615-1.shtml" target="_blank">血泪控诉，请浙江杭州萧山两位书记！还我母亲！</a>
</td>
<td><a class="author" href="http://www.tianya.cn/135654766" target="_blank">风非柏溢</a></td>
<td>66109</td>
<td>889</td>
<td title="null">04-03 11:46</td>


In [202]:
td = articles[3].find_all('td')

In [203]:
td[0]

<td class="td-title ">
<span class="face" title="">
</span>
<a href="/post-free-2849477-1.shtml" target="_blank">
							  【民间语文第161期】 宁波px启示:船进港湾人应上岸<span class="art-ico art-ico-3" title="内有0张图片"></span>
</a>
</td>

In [184]:
td[0].text

'\n\n血泪控诉，请浙江杭州萧山两位书记！还我母亲！\n'

In [185]:
td[0].text.strip()

'血泪控诉，请浙江杭州萧山两位书记！还我母亲！'

In [186]:
td[0].a['href']

'http://bei.tianya.cn/wallet/zuan.do'

In [187]:
td[0]

<td class="td-title facediamond">
<a class="diamond" href="http://bei.tianya.cn/wallet/zuan.do" target="_blank" title="钻石榜"></a>
<a class="" href="http://bbs.tianya.cn/post-free-5876615-1.shtml" target="_blank">血泪控诉，请浙江杭州萧山两位书记！还我母亲！</a>
</td>

In [189]:
td[0].find('a', {'class', ""})['href']

'http://bbs.tianya.cn/post-free-5876615-1.shtml'

In [190]:
td[1]

<td><a class="author" href="http://www.tianya.cn/135654766" target="_blank">风非柏溢</a></td>

In [191]:
td[2]

<td>66109</td>

In [192]:
td[3]

<td>889</td>

In [193]:
td[4]

<td title="null">04-03 11:46</td>

In [205]:
records = []
for k, i in enumerate(articles[1:]):
    td = i.find_all('td')
    title = td[0].text.strip()
    try:
        title_url = td[0].find('a', {'class', ""})['href'] 
    except:
        title_url = td[0].a['href']
    author = td[1].text
    author_url = td[1].a['href']
    views = td[2].text
    replies = td[3].text
    date = td[4]['title']
    record = title + '\t' + title_url+ '\t' + author + '\t'+ author_url + '\t' + views+ '\t'  + replies+ '\t'+ date
    records.append(record)

In [207]:
print(records[0])

血泪控诉，请浙江杭州萧山两位书记！还我母亲！	http://bbs.tianya.cn/post-free-5876615-1.shtml	风非柏溢	http://www.tianya.cn/135654766	66109	889	null


In [208]:
print(records[2])

【民间语文第161期】 宁波px启示:船进港湾人应上岸	/post-free-2849477-1.shtml	贾也	http://www.tianya.cn/50499450	195119	2704	2012-10-29 07:59


# 抓取天涯论坛PX帖子列表

回帖网络（Thread network）的结构
- 列表
- 主帖
- 回帖

In [45]:
articles[2]

<tr>
<td class="td-title facediamond">
<a class="diamond" href="http://bei.tianya.cn/wallet/zuan.do" target="_blank" title="钻石榜"></a>
<a class="" href="http://bbs.tianya.cn/post-free-5881411-1.shtml" target="_blank">检举揭发现任中国石油天然气集团公司董事长、党组书记王宜林行贿受贿</a>
</td>
<td><a class="author" href="http://www.tianya.cn/135764138" target="_blank">stevenxie871130</a></td>
<td>5591</td>
<td>34</td>
<td title="null">04-03 11:24</td>
</tr>

In [262]:
def crawler(page_num, file_name):
    try:
        # open the browser
        url = "http://bbs.tianya.cn/list.jsp?item=free&nextid=%d&order=8&k=PX" % page_num
        content = requests.get(url).text #获取网页的html文本
        soup = BeautifulSoup(content, "lxml") 
        articles = soup.find_all('tr')
        # write down info
        for i in articles[1:]:
            td = i.find_all('td')
            title = td[0].text.strip()
            title_url = td[0].a['href']
            author = td[1].text
            author_url = td[1].a['href']
            views = td[2].text
            replies = td[3].text
            date = td[4]['title']
            record = title + '\t' + title_url+ '\t' + author + '\t'+ \
                        author_url + '\t' + views+ '\t'  + replies+ '\t'+ date
            with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                        p.write(record+"\n") ##!!encode here to utf-8 to avoid encoding

    except Exception as e:
        print(e)
        pass

In [263]:
# crawl all pages
for page_num in range(10):
    print(page_num) 
    crawler(page_num, '../data/tianya_bbs_threads_list2018.txt') 

0
1
2
3
4
5
6
7
8
9


In [46]:
import pandas as pd

df = pd.read_csv('../data/tianya_bbs_threads_list2018.txt',\
                 sep = "\t",  names = ['title', 'link', 'author', \
                 'author_page', 'click', 'reply', 'time'])
df[:3] 

,title,link,author,author_page,click,reply,time
0,血泪控诉，请浙江杭州萧山两位书记！还我母亲！,http://bei.tianya.cn/wallet/zuan.do,风非柏溢,http://www.tianya.cn/135654766,65957,888,null
1,检举揭发现任中国石油天然气集团公司董事长、党组书记王宜林行贿受贿,http://bei.tianya.cn/wallet/zuan.do,stevenxie871130,http://www.tianya.cn/135764138,5591,34,null
2,【民间语文第161期】 宁波px启示:船进港湾人应上岸,/post-free-2849477-1.shtml,贾也,http://www.tianya.cn/50499450,195119,2704,2012-10-29 07:59


In [47]:
len(df)

464

In [48]:
len(df.link)

464

# 抓取作者信息

In [49]:
df.author_page[:5]

0    http://www.tianya.cn/135654766
1    http://www.tianya.cn/135764138
2     http://www.tianya.cn/50499450
3     http://www.tianya.cn/36959960
4     http://www.tianya.cn/53134970
Name: author_page, dtype: object

http://www.tianya.cn/62237033
    
http://www.tianya.cn/67896263
    

In [23]:
# user_info

In [135]:
url = df.author_page[10]
content = requests.get(url).text #获取网页的html文本
soup = BeautifulSoup(content, "lxml") 

In [136]:
soup

<!DOCTYPE HTML>
<!--[if IE 6]><html class="ie6 lte9 lte8 lte7" lang="zh-cn"><![endif]--><!--[if IE 8]><html class="ie8 lte9 lte8" lang="zh-cn"><![endif]--><!--[if IE 9]><html class="ie9 lte9" lang="zh-cn"><![endif]--><!--[if IE 7]><html class="ie7 lte9 lte8 lte7" lang="zh-cn"><![endif]--><!--[if !(IE 6) | !(IE 7) | !(IE 8) | !(IE 9)  ]><!--><html lang="zh-cn"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<title>扬子鱼儿_天涯社区</title>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="Thu, 01 Nov 2012 10:00:00 GMT" http-equiv="Expires"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<link href="http://static.tianyaui.com/global/ty/TY.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/global/my/web/static/css/main_a23fc96.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/global/my/web/static/css/t_ff5e161.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/favicon.ico" rel="shortcut icon"

In [124]:
# user_info = soup.find('div',  {'class', 'userinfo'})('p')
# for i in user_info:
#     print(i.get_text()[4:])

In [125]:
# user_info

In [126]:
user_info = soup.find('div',  {'class', 'userinfo'})('p')
score, reg_time = [i.get_text()[4:] for i in user_info]
print(score, reg_time )

link_info = soup.find_all('div', {'class', 'link-box'})
followed_num, fans_num = [i.a.text for i in link_info]
print(followed_num, fans_num)

293212 2011-04-14 20:49:00
152 28002


In [130]:
activity = soup.find_all('span', {'class', 'subtitle'})
post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
print(post_num, reply_num)

1 12


In [131]:
activity[0]

<span class="subtitle"><a href="http://www.tianya.cn/50499450/group?t=post" target="_blank">主帖1</a>　<a href="http://www.tianya.cn/50499450/group?t=reply" target="_blank">回帖12</a></span>

In [132]:
link_info = soup.find_all('div', {'class', 'link-box'})
followed_num, fans_num = [i.a.text for i in link_info]
print(followed_num, fans_num)

152 28002


In [133]:
link_info[0].a.text

'152'

In [101]:
# user_info = soup.find('div',  {'class', 'userinfo'})('p')
# user_infos = [i.get_text()[4:] for i in user_info]
            
def author_crawler(url, file_name):
    try:
        content = requests.get(url).text #获取网页的html文本
        soup = BeautifulSoup(content, "lxml")
        link_info = soup.find_all('div', {'class', 'link-box'})
        followed_num, fans_num = [i.a.text for i in link_info]
        try:
            activity = soup.find_all('span', {'class', 'subtitle'})
            post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
        except:
            post_num, reply_num = '1', '0'
        record =  '\t'.join([url, followed_num, fans_num, post_num, reply_num])
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record+"\n") ##!!encode here to utf-8 to avoid encoding

    except Exception as e:
        print(e, url)
        record =  '\t'.join([url, 'na', 'na', 'na', 'na'])
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    p.write(record+"\n") ##!!encode here to utf-8 to avoid encoding
        pass

In [134]:
soup

<!DOCTYPE HTML>
<!--[if IE 6]><html class="ie6 lte9 lte8 lte7" lang="zh-cn"><![endif]--><!--[if IE 8]><html class="ie8 lte9 lte8" lang="zh-cn"><![endif]--><!--[if IE 9]><html class="ie9 lte9" lang="zh-cn"><![endif]--><!--[if IE 7]><html class="ie7 lte9 lte8 lte7" lang="zh-cn"><![endif]--><!--[if !(IE 6) | !(IE 7) | !(IE 8) | !(IE 9)  ]><!--><html lang="zh-cn"><!--<![endif]-->
<head>
<meta charset="utf-8"/>
<title>贾也_天涯社区</title>
<meta content="no-cache" http-equiv="Pragma"/>
<meta content="Thu, 01 Nov 2012 10:00:00 GMT" http-equiv="Expires"/>
<meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
<link href="http://static.tianyaui.com/global/ty/TY.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/global/my/web/static/css/main_a23fc96.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/global/my/web/static/css/t_ff5e161.css" rel="stylesheet" type="text/css"/>
<link href="http://static.tianyaui.com/favicon.ico" rel="shortcut icon" t

In [137]:
url = df.author_page[10]
content = requests.get(url).text #获取网页的html文本
soup = BeautifulSoup(content, "lxml")
link_info = soup.find_all('div', {'class', 'link-box'})
followed_num, fans_num = [i.a.text for i in link_info]
try:
    activity = soup.find_all('span', {'class', 'subtitle'})
    post_num, reply_num = [j.text[2:] for i in activity[:1] for j in i('a')]
except:
    post_num, reply_num = '1', '0'
record =  '\t'.join([url, followed_num, fans_num, post_num, reply_num])


In [74]:
import sys
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

In [140]:
import time, random
for k, url in enumerate(df.author_page[:15]):
    time.sleep(random.random()*10) # 天涯存在反抓取机制，需要降低抓取速度！
    flushPrint(k)
    author_crawler(url, '../data/tianya_bbs_threads_author_info2018.txt') 

14

http://www.tianya.cn/50499450/follow  

还可抓取他们的关注列表和粉丝列表

***
***
# 数据抓取：
> # 使用Python抓取回帖
***
***

王成军

wangchengjun@nju.edu.cn

计算传播网 http://computational-communication.com

In [209]:
df.link[2]

'/post-free-2849477-1.shtml'

In [211]:
url = 'http://bbs.tianya.cn' + df.link[2]
url

'http://bbs.tianya.cn/post-free-2849477-1.shtml'

In [264]:
from IPython.display import display_html, HTML
HTML('<iframe src=http://bbs.tianya.cn/post-free-2849477-1.shtml width=1000 height=500></iframe>')
# the webpage we would like to crawl

In [215]:
post = requests.get(url).text #获取网页的html文本  
post_soup = BeautifulSoup(post, "lxml") 
#articles = soup.find_all('tr')

In [216]:
print (post_soup.prettify())[:5000]

<!DOCTYPE HTML>
<html>
 <head>
  <meta charset="utf-8"/>
  <title>
   【民间语文第161期】宁波px启示：船进港湾人应上岸_天涯杂谈_论坛_天涯社区
  </title>
  <meta content=".marg {width:100%; background:#E8E3CF; overflow:hidden; margin:0 auto; padding-top:10px; padding-left:0px;padding-right: 0px;padding-bottom:10px;}.ztpage {BORDER-RIGHT: #dcc696 3px solid; PADDING-RIGHT: 5px; BORDER-TOP: #dcc696 3px solid; ..." itemprop="description" name="description"/>
  <meta content="【民间语文第161期】宁波px启示：船进港湾人应上岸_天涯杂谈_论坛_天涯社区" itemprop="name"/>
  <meta content="" itemprop="image"/>
  <meta content="IE=Edge" http-equiv="X-UA-Compatible"/>
  <meta content="贾也" name="author"/>
  <meta content="no-transform" http-equiv="Cache-Control"/>
  <meta content="no-siteapp" http-equiv="Cache-Control"/>
  <meta content="pc" name="applicable-device"/>
  <meta content="format=xhtml; url=http://bbs.tianya.cn/m/post-free-2849477-1.shtml" name="mobile-agent"/>
  <meta content="format=html5; url=http://bbs.tianya.cn/m/post-free-2849477-1.shtml" name="mo

TypeError: 'NoneType' object is not subscriptable

In [217]:
pa = post_soup.find_all('div', {'class', 'atl-item'})
len(pa)

91

In [219]:
print(pa[0]) 

<div _host="%E8%B4%BE%E4%B9%9F" _hostid="50499450" class="atl-item host-item">
<div class="atl-content">
<div class="atl-con-hd clearfix">
<div class="atl-con-hd-l"></div>
<div class="atl-con-hd-r"></div>
</div>
<div class="atl-con-bd clearfix">
<div class="bbs-content clearfix">
<style>.marg {width:100%; background:#E8E3CF; overflow:hidden; margin:0 auto; padding-top:10px; padding-left:0px;padding-right: 0px;padding-bottom:10px;}.ztpage {BORDER-RIGHT: #dcc696 3px solid; PADDING-RIGHT: 5px; BORDER-TOP: #dcc696 3px solid; PADDING-LEFT: 5px; PADDING-BOTTOM: 5px; MARGIN: 0px auto; BORDER-LEFT: #dcc696 3px solid; WIDTH: 880px; PADDING-TOP: 5px; BORDER-BOTTOM: #dcc696 3px solid; BACKGROUND-COLOR: #dededd;}.ywdu{width:880px;}.ywdu .dao-1{float:left;width:500px; height: 370px}.ywdu .dao-1t{font-size:18px; font-family:微软雅黑; color:#ff0000;padding:6px 15px;float:left;width:460px;padding-left:215px; padding-top:18px;padding-right:2px}.ywdu .dao-1m{font-size:25px; font-family:微软雅黑,黑体; color:#90060

In [220]:
pa[1]

<div _host="三平67" _hostid="22122799" class="atl-item" id="1" js_restime="2012-10-29 08:13:54" js_username="三平67" replyid="92785089">
<div class="atl-head" id="0de15aa9a1c8d8b3b7d9c6b952585c67">
<div class="atl-head-reply"></div>
<div class="atl-info">
<span>作者：<a class="js-vip-check" href="http://www.tianya.cn/22122799" target="_blank" uid="22122799" uname="三平67">三平67</a> </span>
<span>时间：2012-10-29 08:13:54</span>
</div>
</div>
<div class="atl-content">
<div class="atl-con-hd clearfix">
<div class="atl-con-hd-l"></div>
<div class="atl-con-hd-r"></div>
</div>
<div class="atl-con-bd clearfix">
<div class="bbs-content">
							　　我们中国人都在一条船，颠簸已久，我们都想做宁波人，希望有一个风平浪静的港湾，面朝大海，春暖花开！<br/>
<br/>
　　.<a href="http://www4.tianya.cn/new/publicforum/Content.asp?strItem=free&amp;idArticle=2638338&amp;flag=1" target="_blank"><font color="blue"><b>[《民间语文》往期回顾 ]</b></font></a><br/>
<br/>
<br/>
</div>
<div class="atl-reply">
<a author="三平67" authorid="22122799" class="reportme a-link" href="javascript:voi

In [221]:
pa[89]

<div _host="迷惘的90后" _hostid="62943356" class="atl-item" id="99" js_restime="2012-10-29 12:30:46" js_username="迷惘的90后" replyid="92785324">
<div class="atl-head" id="2bb7be822c18540ab1b041fbf06819eb">
<div class="atl-head-reply"></div>
<div class="atl-info">
<span>作者：<a class="js-vip-check" href="http://www.tianya.cn/62943356" target="_blank" uid="62943356" uname="迷惘的90后">迷惘的90后</a> </span>
<span>时间：2012-10-29 12:30:46</span>
</div>
</div>
<div class="atl-content">
<div class="atl-con-hd clearfix">
<div class="atl-con-hd-l"></div>
<div class="atl-con-hd-r"></div>
</div>
<div class="atl-con-bd clearfix">
<div class="bbs-content">
							　　我们中国人都在一条船，颠簸已久，我们都想做宁波人，希望有一个风平浪静的港湾，面朝大海，春暖花开！
							
							
						</div>
<div class="atl-reply">
<a author="迷惘的90后" authorid="62943356" class="reportme a-link" href="javascript:void(0);" replyid="92785324" replytime="2012-10-29 12:30:46">举报</a> |
														
							<span>99楼</span> |
							<a class="a-link-2 ir-shang" floor="99" href="javascript

作者：柠檬在追逐 时间：2012-10-28 21:33:55

　　@lice5 2012-10-28 20:37:17
  
　　作为宁波人 还是说一句：革命尚未成功 同志仍需努力 
  
　　-----------------------------
  
　　对 现在说成功还太乐观，就怕说一套做一套

作者：lice5 时间：2012-10-28 20:37:17
        
　　作为宁波人 还是说一句：革命尚未成功 同志仍需努力

4	/post-free-4242156-1.shtml	2014-04-09 15:55:35	61943225	野渡自渡人	@Y雷政府34楼2014-04-0422:30:34　　野渡君雄文！支持是必须的。　　-----------------------------　　@清坪过客16楼2014-04-0804:09:48　　绝对的权力导致绝对的腐败！　　-----------------------------　　@T大漠鱼T35楼2014-04-0810:17:27　　@周丕东@普欣@拾月霜寒2012@小摸包@姚文嚼字@四號@凌宸@乔志峰@野渡自渡人@曾兵2010@缠绕夜色@曾颖@风青扬请关注

In [233]:
pa[0].find('div', {'class', 'bbs-content'}).text.strip()

'.marg {width:100%; background:#E8E3CF; overflow:hidden; margin:0 auto; padding-top:10px; padding-left:0px;padding-right: 0px;padding-bottom:10px;}.ztpage {BORDER-RIGHT: #dcc696 3px solid; PADDING-RIGHT: 5px; BORDER-TOP: #dcc696 3px solid; PADDING-LEFT: 5px; PADDING-BOTTOM: 5px; MARGIN: 0px auto; BORDER-LEFT: #dcc696 3px solid; WIDTH: 880px; PADDING-TOP: 5px; BORDER-BOTTOM: #dcc696 3px solid; BACKGROUND-COLOR: #dededd;}.ywdu{width:880px;}.ywdu .dao-1{float:left;width:500px; height: 370px}.ywdu .dao-1t{font-size:18px; font-family:微软雅黑; color:#ff0000;padding:6px 15px;float:left;width:460px;padding-left:215px; padding-top:18px;padding-right:2px}.ywdu .dao-1m{font-size:25px; font-family:微软雅黑,黑体; color:#90060B;font-weight:bold; padding:6px 15px;float:left;width:460px;padding-left:30px; padding-top:40px;padding-right:10px}.ywdu .dao-1b{font-size:14px; font-family:微软雅黑,黑体; color:#fff; line-height:16px;float:left;width:460px;height: 238px;padding-left:10px; padding-top:30px;padding-right:180px

In [230]:
pa[0].find('div', {'class', 'marg'}).text.strip()

'第161期导语：人人宁波，面朝大海，春暖花开!\n\u3000\u3000宁波的事，怎谈？无从谈，太凝重！从南的厦门，到北的大连，再到港通天下的宁波，一而再，再而三，复制再复制。我们的政府从这些事件中得到什么教训？即使是血的教训，血是民众的，教训却教到爪哇岛去了。中国，一如既往，上街表达诉求依然用"散步"代称；一如既往，"散步"依然面对全副武装的警力；一如既往，关于"散步"的定论依然是危害社会稳定的论调……乌坎是个村，什邡是个县，启东是个县级市，宁波是个副省级城市，他们都属于中国，而那里的人们次第地醒来，政府是否依然沉睡？是否还是在装睡，注定无法喊醒？\n\u3000\u3000虽然宁波不宁，镇海被镇，中国发展陷入了困境，但是我们梦想依然丰满，希望人人宁波，面朝大海，春暖花开，希望所有的人依然同舟共济，总想驶进属于大家的港湾，中国歇歇脚！\n我爱宁波，我爱青山绿水一、怪味的中国：请温柔些我们要生存，我们要活命\u3000\u3000从厦门到大连，再到宁波，PX每到一处都激起民意强烈的反弹，为什么政府还是执迷不悟地做？我们似乎解读到了一个前所未有的傲慢的政府，或者说屁股严重放错的政府。\n\u3000\u3000就今年而言，从什邡到启东，再到宁波，因环保项目引发民众上街，原因却是一脉相承的。这些地方的民众都按照法律的规定和程序，向法院提起了诉讼，公民原本是理智和理性的。但地方化的司法却不予受理，强行把民众推上了街头。司法失效、司法地方化、司法没有独立性，就必然使社会走上失序紊乱的轨道。公平正义不彰，国家只有向下沉沦，而人们维护自身的生存权，却徒剩下对抗的途径。谁之罪？问苍天，苍天不语！ \n\u3000\u3000说句实在话，我越来越看不懂了，应该说，我对于祖国的感情，很纯粹，深深地爱，却深深地痛。这回，甚至连我爱人因为宁波这个事件很受伤，她是单纯的教师，"微博控"才2个月，一直处于CCTV状态，没见多少世面，看到宁波\'散步\'就发生不远的地方，偶尔看到点流血，便心里难受到哭的程度，质问我政府怎么能这样？\n\u3000\u3000我劝她说，习惯就好，今年就有什邡、启东，周边就有织里、海宁！见多就不怪了。除此之外，我觉得任何语言都肌无力。难道我可以规定她只能看CCTV，跟随央视的"幸福调查"，在催眠中入梦？再说我真没"跨省销报"的霸气，为照顾她的"三观"，去关停新

In [232]:
pa[87].find('div', {'class', 'bbs-content'}).text.strip()

'生不逢国'

In [224]:
pa[1].a 

<a class="js-vip-check" href="http://www.tianya.cn/22122799" target="_blank" uid="22122799" uname="三平67">三平67</a>

In [225]:
pa[0].find('a', class_ = 'reportme a-link')

<a author="贾也" authorid="50499450" class="reportme a-link" href="javascript:void(0);" replyid="0" replytime="2012-10-29 07:59:00"> 举报</a>

In [226]:
pa[0].find('a', class_ = 'reportme a-link')['replytime']

'2012-10-29 07:59:00'

In [227]:
pa[0].find('a', class_ = 'reportme a-link')['author']

'贾也'

In [235]:
for i in pa[:10]:
    p_info = i.find('a', class_ = 'reportme a-link')
    p_time = p_info['replytime']
    p_author_id = p_info['authorid']
    p_author_name = p_info['author']
    p_content = i.find('div', {'class', 'bbs-content'}).text.strip()
    p_content = p_content.replace('\t', '')
    print(p_time, '--->', p_author_id, '--->', p_author_name,'--->', p_content, '\n')

2012-10-29 07:59:00 ---> 50499450 ---> 贾也 ---> .marg {width:100%; background:#E8E3CF; overflow:hidden; margin:0 auto; padding-top:10px; padding-left:0px;padding-right: 0px;padding-bottom:10px;}.ztpage {BORDER-RIGHT: #dcc696 3px solid; PADDING-RIGHT: 5px; BORDER-TOP: #dcc696 3px solid; PADDING-LEFT: 5px; PADDING-BOTTOM: 5px; MARGIN: 0px auto; BORDER-LEFT: #dcc696 3px solid; WIDTH: 880px; PADDING-TOP: 5px; BORDER-BOTTOM: #dcc696 3px solid; BACKGROUND-COLOR: #dededd;}.ywdu{width:880px;}.ywdu .dao-1{float:left;width:500px; height: 370px}.ywdu .dao-1t{font-size:18px; font-family:微软雅黑; color:#ff0000;padding:6px 15px;float:left;width:460px;padding-left:215px; padding-top:18px;padding-right:2px}.ywdu .dao-1m{font-size:25px; font-family:微软雅黑,黑体; color:#90060B;font-weight:bold; padding:6px 15px;float:left;width:460px;padding-left:30px; padding-top:40px;padding-right:10px}.ywdu .dao-1b{font-size:14px; font-family:微软雅黑,黑体; color:#fff; line-height:16px;float:left;width:460px;height: 238px;padding-l

# 如何翻页

http://bbs.tianya.cn/post-free-2848797-1.shtml
    
    http://bbs.tianya.cn/post-free-2848797-2.shtml
        
        http://bbs.tianya.cn/post-free-2848797-3.shtml

In [236]:
post_soup.find('div', {'class', 'atl-pages'})#.['onsubmit']

<div class="atl-pages"><form action="" method="get" onsubmit="return goPage(this,'free',2849477,28);">
<span>上页</span>
<strong>1</strong>
<a href="/post-free-2849477-2.shtml">2</a>
<a href="/post-free-2849477-3.shtml">3</a>
<a href="/post-free-2849477-4.shtml">4</a>
…
<a href="/post-free-2849477-28.shtml">28</a>
<a class="js-keyboard-next" href="/post-free-2849477-2.shtml">下页</a>
 到<input class="pagetxt" name="page" type="text"/>页 <input class="pagebtn" maxlength="6" name="gopage" type="submit" value="确定"/></form></div>

In [237]:
post_pages = post_soup.find('div', {'class', 'atl-pages'})
post_pages = post_pages.form['onsubmit'].split(',')[-1].split(')')[0]
post_pages
#post_soup.select('.atl-pages')[0].select('form')[0].select('onsubmit')


'28'

In [238]:
url = 'http://bbs.tianya.cn' + df.link[2]
url_base = ''.join(url.split('-')[:-1]) + '-%d.shtml'
url_base

'http://bbs.tianya.cn/postfree2849477-%d.shtml'

In [239]:
def parsePage(pa):
    records = []
    for i in pa:
        p_info = i.find('a', class_ = 'reportme a-link')
        p_time = p_info['replytime']
        p_author_id = p_info['authorid']
        p_author_name = p_info['author']
        p_content = i.find('div', {'class', 'bbs-content'}).text.strip()
        p_content = p_content.replace('\t', '').replace('\n', '')#.replace(' ', '')
        record = p_time + '\t' + p_author_id+ '\t' + p_author_name + '\t'+ p_content
        records.append(record)
    return records

import sys
def flushPrint(s):
    sys.stdout.write('\r')
    sys.stdout.write('%s' % s)
    sys.stdout.flush()

In [241]:
url_1 = 'http://bbs.tianya.cn' + df.link[10]
content = requests.get(url_1).text #获取网页的html文本
post_soup = BeautifulSoup(content, "lxml") 
pa = post_soup.find_all('div', {'class', 'atl-item'})
b = post_soup.find('div', class_= 'atl-pages')
b

<div class="atl-pages host-pages"></div>

In [244]:
url_0 = 'http://bbs.tianya.cn' + df.link[2]
content = requests.get(url_0).text #获取网页的html文本
post_soup = BeautifulSoup(content, "lxml") 
pa = post_soup.find_all('div', {'class', 'atl-item'})
a = post_soup.find('div', class_= 'atl-pages')
a

<div class="atl-pages"><form action="" method="get" onsubmit="return goPage(this,'free',2849477,28);">
<span>上页</span>
<strong>1</strong>
<a href="/post-free-2849477-2.shtml">2</a>
<a href="/post-free-2849477-3.shtml">3</a>
<a href="/post-free-2849477-4.shtml">4</a>
…
<a href="/post-free-2849477-28.shtml">28</a>
<a class="js-keyboard-next" href="/post-free-2849477-2.shtml">下页</a>
 到<input class="pagetxt" name="page" type="text"/>页 <input class="pagebtn" maxlength="6" name="gopage" type="submit" value="确定"/></form></div>

In [245]:
a.form

<form action="" method="get" onsubmit="return goPage(this,'free',2849477,28);">
<span>上页</span>
<strong>1</strong>
<a href="/post-free-2849477-2.shtml">2</a>
<a href="/post-free-2849477-3.shtml">3</a>
<a href="/post-free-2849477-4.shtml">4</a>
…
<a href="/post-free-2849477-28.shtml">28</a>
<a class="js-keyboard-next" href="/post-free-2849477-2.shtml">下页</a>
 到<input class="pagetxt" name="page" type="text"/>页 <input class="pagebtn" maxlength="6" name="gopage" type="submit" value="确定"/></form>

In [247]:
if b.form:
    print('true')
else:
    print('false')

false


In [251]:
import random
import time

def crawler(url, file_name):
    try:
        # open the browser
        url_1 = 'http://bbs.tianya.cn' + url
        content = requests.get(url_0).text #获取网页的html文本
        post_soup = BeautifulSoup(content, "lxml") 
        # how many pages in a post
        post_form = post_soup.find('div', {'class', 'atl-pages'})
        if post_form.form:
            post_pages = post_form.form['onsubmit'].split(',')[-1].split(')')[0]
            post_pages = int(post_pages)
            url_base = '-'.join(url_1.split('-')[:-1]) + '-%d.shtml'
        else:
            post_pages = 1
        # for the first page
        pa = post_soup.find_all('div', {'class', 'atl-item'})
        records = parsePage(pa)
        with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
            for record in records:    
                p.write('1'+ '\t' + url + '\t' + record+"\n") 
        # for the 2nd+ pages
        if post_pages > 1:
            for page_num in range(2, post_pages+1):
                time.sleep(random.random())
                flushPrint(page_num)
                url2 =url_base  % page_num
                content = requests.get(url2).text #获取网页的html文本
                post_soup = BeautifulSoup(content, "lxml") 
                pa = post_soup.find_all('div', {'class', 'atl-item'})
                records = parsePage(pa)
                with open(file_name,'a') as p: # '''Note'''：Ａppend mode, run only once!
                    for record in records:    
                        p.write(str(page_num) + '\t' +url + '\t' + record+"\n") 
        else:
            pass
    except Exception as e:
        print(e)
        pass

# 测试

In [252]:
url = df.link[2]
file_name = '../data/tianya_bbs_threads_4test.txt'
crawler(url, file_name)

28

# 正式抓取！

In [108]:
for k, link in enumerate(df.link):
    flushPrint(link)
    if k % 10== 0:
        print 'This it the post of : ' + str(k)
    file_name = '../data/tianya_bbs_threads_network_2017.txt'
    crawler(link, file_name)

/post-free-2839539-1.shtmlThis it the post of : 0
/post-free-923115-1.shtmlThis it the post of : 10
/post-free-944280-1.shtmlThis it the post of : 20
/post-free-3324751-1.shtmlThis it the post of : 30
/post-free-3361796-1.shtmlThis it the post of : 40
/post-free-3533685-1.shtmlThis it the post of : 50
/post-free-4242705-1.shtmlThis it the post of : 60
/post-free-3291045-1.shtmlThis it the post of : 70
/post-free-1066741-1.shtmlThis it the post of : 80
/post-free-3312925-1.shtmlThis it the post of : 90
/post-free-3834860-1.shtmlThis it the post of : 100
/post-free-1079222-1.shtmlThis it the post of : 110
/post-free-4236159-1.shtmlThis it the post of : 120
/post-free-3282507-1.shtmlThis it the post of : 130
/post-free-3394373-1.shtmlThis it the post of : 140
/post-free-3352554-1.shtmlThis it the post of : 150
/post-free-3294035-1.shtmlThis it the post of : 160
/post-free-3305558-1.shtmlThis it the post of : 170
/post-free-3228423-1.shtmlThis it the post of : 180
/post-free-3422932-1.shtm

# 读取数据

In [253]:
dtt = []
with open('../data/tianya_bbs_threads_network_2017.txt', 'r') as f:
    for line in f:
        pnum, link, time, author_id, author, content = line.replace('\n', '').split('\t')
        dtt.append([pnum, link, time, author_id, author, content])
len(dtt)

5277

In [254]:
dt = pd.DataFrame(dtt)
dt[:5]

,0,1,2,3,4,5
0,1,/post-free-2839539-1.shtml,2012-10-24 12:41:00,74341835,无上卫士ABC,2012-10-24 10:42:47 来源: 人民网(北京) 核心提示：10月初，宁波...
1,1,/post-free-2839539-1.shtml,2012-10-24 13:26:06,74422795,16号渔政船,对，当官的住两年试试我们就信了
2,1,/post-free-2839539-1.shtml,2012-10-24 14:02:37,15068828,fossil2918,广大网民理性表达网络民意，不传播、不轻信网上不实信息，共同维护社会和谐稳定大局。
3,1,/post-free-2839539-1.shtml,2012-10-25 14:25:31,75446299,一言难尽XCSTZ,这样啊
4,1,/post-free-2839539-1.shtml,2012-10-25 14:49:59,18135446,owekoker07298,关注


In [255]:
dt=dt.rename(columns = {0:'page_num', 1:'link', 2:'time', 3:'author',4:'author_name', 5:'reply'})
dt[:5]

,page_num,link,time,author,author_name,reply
0,1,/post-free-2839539-1.shtml,2012-10-24 12:41:00,74341835,无上卫士ABC,2012-10-24 10:42:47 来源: 人民网(北京) 核心提示：10月初，宁波...
1,1,/post-free-2839539-1.shtml,2012-10-24 13:26:06,74422795,16号渔政船,对，当官的住两年试试我们就信了
2,1,/post-free-2839539-1.shtml,2012-10-24 14:02:37,15068828,fossil2918,广大网民理性表达网络民意，不传播、不轻信网上不实信息，共同维护社会和谐稳定大局。
3,1,/post-free-2839539-1.shtml,2012-10-25 14:25:31,75446299,一言难尽XCSTZ,这样啊
4,1,/post-free-2839539-1.shtml,2012-10-25 14:49:59,18135446,owekoker07298,关注


In [256]:
dt.reply[:100]

0     2012-10-24 10:42:47　来源: 人民网(北京)　　　核心提示：10月初，宁波...
1                                       对，当官的住两年试试我们就信了
2              广大网民理性表达网络民意，不传播、不轻信网上不实信息，共同维护社会和谐稳定大局。
3                                                   这样啊
4                                                    关注
5     水浒一块钱一本买一送三　　水浒一块钱一本买一送三　　水浒一块钱一本买一送三　　水浒一块钱一本...
6                                        我们不要化工，我们要净土！！
7                             我在八卦发的帖子被封口，不能留言，连看都不能看！！
8     @想吃甜枣 22楼 　　PX，请你滚出宁波！！　　---------------------...
9     @fossil2918 3楼 　　广大网民理性表达网络民意，不传播、不轻信网上不实信息，共同...
10            政府引进这种类似项目，我们知情么？有选择么？不过日本人侵华，也不会经过中国人同意…
11    镇海中学领导对老师已经下达了禁言令：好好教书上课，其他的不该管的不要管，不该说的不要说，学校...
12                               已经禁止镇海私家车开进市区了，违者扣12分！
13    回复第54楼(作者:@小桃子mama 于 2012-10-26 10:07) 　　作者：一笑...
14                      宁波人过几年要绝种了，水和空气都是有毒的，生出来的孩子崎形的。
15                                       江苏上支持你们一定要抗争到底
16                                            光上网叫唤有鸡吧用
17                                            湖北

## 总帖数是多少？

http://search.tianya.cn/bbs?q=PX  共有18459 条内容

In [93]:
18459/50

369

实际上到第10页就没有了 http://bbs.tianya.cn/list.jsp?item=free&order=1&nextid=9&k=PX, 原来那只是天涯论坛，还有其它各种版块，如天涯聚焦：  http://focus.tianya.cn/  等等。

- 娱乐八卦 512
- 股市论坛 187
- 情感天地 242
- 天涯杂谈 1768 

在天涯杂谈搜索雾霾，有41页 http://bbs.tianya.cn/list.jsp?item=free&order=20&nextid=40&k=%E9%9B%BE%E9%9C%BE

# 天涯SDK

http://open.tianya.cn/wiki/index.php?title=SDK%E4%B8%8B%E8%BD%BD